In [ ]:
# main training data set creation script
# requires grayscale, single layer images in .tiff format
# 1. create_split randomly splits Montages in train, test (cross_validation=True) or train, test, validation sets (cross_validation=False)
# 2. erode_dataset erodes images (default iterations=64)
# 3. crop_dataset crops images into pataches. Locations of patches is randomized. Default size is 256*256.

import sys
sys.path.append("..")

from utils.data import create_split, erode_dataset, crop_dataset, equalize_dataset

drive = "D"

dataset_id = "20230301"

# 1. Create a split into train/test images
create_split(f"{drive}:/Data/Montagen_3x3_ConeMaps", f"{drive}:/Data/ConeDatasets/{dataset_id}_cross")
#create_split("D:/MatLab/CM_Alex_git/TestTraining/data", "D:/MatLab/CM_Alex_git/TestTraining/test")

# 2. Erode images to counteract imaging artefacts (and missing GT cones for BAK8095L!)
erode_dataset(f"{drive}:/Data/ConeDatasets/{dataset_id}_cross", f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded")
#erode_dataset("D:/MatLab/CM_Alex_git/TestTraining/test", "D:/MatLab/CM_Alex_git/TestTraining/test_eroded")

# 3. Locally equalize the histogram of the images
# equalize_dataset(f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded", f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded_equalized_64", radius=64)

# 4. Create crops to use during training
crop_dataset(f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded", f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded_cropped")
#crop_dataset("D:/MatLab/CM_Alex_git/TestTraining/test_eroded", "D:/MatLab/CM_Alex_git/TestTraining/test_eroded_cropped")

0it [00:00, ?it/s]
2it [00:00, 27.09it/s]
0it [00:00, ?it/s]
2it [00:00,  2.20it/s]


In [ ]:
# Create noisy and blurred datasets (optional)
import sys
sys.path.append("..")

from utils.data import create_split, erode_dataset, crop_dataset, equalize_dataset, blur_dataset, noise_dataset

drive = "D"

dataset_id = "20230301"

# Blur
for sigma in [0.33, 0.67, 1.0, 1.33]:
    blur_dataset(f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded", f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded_blur_{sigma}", sigma=sigma)

# Noise
for sigma in [1, 2, 4, 8]:
    noise_dataset(f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded", f"{drive}:/Data/ConeDatasets/{dataset_id}_cross_eroded_noise_{sigma}", sigma=sigma)

In [ ]:
# Check images GT coverage (optional)
import numpy as np
import matplotlib.pyplot as plt
import os

def overlay_gt(image, label, id):
    gt = np.argwhere(label == 1)
    y, x = gt[:,0], gt[:,1]
    _ = plt.figure(dpi=150)
    plt.title(id)
    plt.imshow(image, cmap="gray")
    plt.scatter(x,y,c="green",s=0.1)
    plt.show()

dataset_id = "20230301_cross_eroded"
#dataset_id = "20230301_cross_eroded_cropped"

for input_path in [f"F:/Data/ConeDatasets/{dataset_id}/test", f"F:/Data/ConeDatasets/{dataset_id}/train"]:
    items = os.listdir(input_path)
    if len(items) > 64:
        items = items[:64]
    subjects = sorted(items)
    n = len(subjects)

    for subject in subjects:
        subject_path = os.path.join(input_path, subject)
        data = np.load(subject_path)
        image, label = data["image"], data["label"]
        overlay_gt(image, label, subject)